In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [3]:
anon = pd.read_csv('../Data_Anom/THE_309', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [4]:
print(anon)

           id                date  latitude  longitude  week
77438     154 2015-04-22 08:24:00     4.865     45.722    17
141649    154 2015-04-26 10:11:00     4.853     45.738    17
216498    194 2015-04-03 23:51:00     4.866     45.756    14
227416     93 2015-03-12 09:40:00     6.057     45.275    11
367474    647 2015-03-31 19:29:00     4.886     45.765    14
...       ...                 ...       ...        ...   ...
34335650  901 2015-04-13 03:10:00     2.281     48.905    16
34372750  693 2015-04-08 11:49:00     4.880     45.785    15
34429162  191 2015-03-07 06:58:00     4.657     46.076    10
34471296  280 2015-03-21 21:59:00     5.053     45.819    12
34528516  612 2015-04-02 11:11:00     4.846     45.752    14

[742 rows x 5 columns]


In [5]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [6]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['1' '100' '1003' '1008' '1010' '1018' '1019' '1021' '1024' '1025' '1026'
 '103' '1033' '1034' '1037' '1040' '1041' '1045' '1047' '1061' '108' '111'
 '114' '117' '118' '119' '122' '132' '133' '134' '135' '137' '138' '140'
 '143' '144' '145' '148' '149' '154' '155' '156' '157' '158' '16' '161'
 '162' '163' '165' '17' '173' '177' '178' '179' '181' '184' '185' '187'
 '188' '190' '191' '194' '196' '197' '198' '2' '200' '202' '203' '204'
 '205' '209' '211' '214' '217' '219' '22' '224' '225' '228' '229' '230'
 '231' '233' '239' '240' '241' '243' '251' '258' '262' '265' '27' '270'
 '274' '280' '284' '285' '289' '29' '290' '303' '307' '308' '309' '31'
 '310' '311' '314' '316' '317' '319' '321' '322' '324' '325' '33

In [7]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [8]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(delta_ratio*delta_time*distance*delta_hour) #*delta_ratio*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 'DEL'
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

195
{'1': {'2015-10': ['944'], '2015-11': ['678'], '2015-12': ['285'], '2015-13': ['913'], '2015-14': ['616'], '2015-15': ['881'], '2015-16': ['230'], '2015-17': ['633'], '2015-18': ['415'], '2015-19': ['338'], '2015-20': ['842']}, '2': {'2015-10': ['952'], '2015-11': ['162'], '2015-12': ['932'], '2015-13': ['807'], '2015-14': ['876'], '2015-15': ['394'], '2015-16': ['310'], '2015-17': ['859'], '2015-18': ['691'], '2015-19': ['560'], '2015-20': ['641']}, '107': {'2015-10': ['944'], '2015-11': ['724'], '2015-12': ['640']}, '98': {'2015-10': ['103'], '2015-11': ['967']}, '21': {'2015-10': ['620'], '2015-11': ['491']}, '5': {'2015-10': ['944'], '2015-11': ['343']}, '9': {'2015-10': ['944'], '2015-11': ['93'], '2015-12': ['285'], '2015-13': ['942'], '2015-14': ['696'], '2015-15': ['1026'], '2015-16': ['98'], '2015-19': ['527']}, '6': {'2015-10': ['404'], '2015-11': ['343'], '2015-12': ['350'], '2015-13': ['942'], '2015-14': ['850'], '2015-15': ['394'], '2015-16': ['608']}, '7': {'2015-10':

In [9]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "944"
        ],
        "2015-11": [
            "678"
        ],
        "2015-12": [
            "285"
        ],
        "2015-13": [
            "913"
        ],
        "2015-14": [
            "616"
        ],
        "2015-15": [
            "881"
        ],
        "2015-16": [
            "230"
        ],
        "2015-17": [
            "633"
        ],
        "2015-18": [
            "415"
        ],
        "2015-19": [
            "338"
        ],
        "2015-20": [
            "842"
        ]
    },
    "2": {
        "2015-10": [
            "952"
        ],
        "2015-11": [
            "162"
        ],
        "2015-12": [
            "932"
        ],
        "2015-13": [
            "807"
        ],
        "2015-14": [
            "876"
        ],
        "2015-15": [
            "394"
        ],
        "2015-16": [
            "310"
        ],
        "2015-17": [
            "859"
        ],
        "2015-18": [